# Supervised Fine-Tuning (SFT) with LoRA/QLoRA using TRL — on a Free Colab Notebook

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/huggingface/trl/blob/main/examples/notebooks/sft_trl_lora_qlora.ipynb)

![trl banner](https://huggingface.co/datasets/trl-lib/documentation-images/resolve/main/trl_banner_dark.png)

Easily fine-tune Large Language Models (LLMs) or Vision-Language Models (VLMs) with **LoRA** or **QLoRA** using the [**Transformers Reinforcement Learning (TRL)**](https://github.com/huggingface/trl) library built by Hugging Face — all within a **free Google Colab notebook** (powered by a **T4 GPU**.).  

- [TRL GitHub Repository](https://github.com/huggingface/trl) — star us to support the project!  
- [Official TRL Examples](https://huggingface.co/docs/trl/example_overview)  
- [Community Tutorials](https://huggingface.co/docs/trl/community_tutorials)

## Key concepts

- **SFT**: Trains models from example input-output pairs to align behavior with human preferences.
- **LoRA**: Updates only a few low-rank parameters, reducing training cost and memory.
- **QLoRA**: A quantized version of LoRA that enables even larger models to fit on small GPUs.
- **TRL**: The Hugging Face library that makes fine-tuning and reinforcement learning simple and efficient.

Learn how to perform **Supervised Fine-Tuning (SFT)** with **LoRA/QLoRA** using **TRL**.

## Install dependencies

We'll install **TRL** with the **PEFT** extra, which ensures all main dependencies such as **Transformers** and **PEFT** (a package for parameter-efficient fine-tuning, e.g., LoRA/QLoRA) are included. Additionally, we'll install **trackio** to log and monitor our experiments, and **bitsandbytes** to enable quantization of LLMs, reducing memory consumption for both inference and training.

In [ ]:
!pip install -Uq "trl[peft]" trackio bitsandbytes liger-kernel wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 18.0 MB/s eta 0:00:00


### Log in to Hugging Face

Log in to your **Hugging Face** account to save your fine-tuned model, track your experiment results directly on the Hub or access gated models. You can find your **access token** on your [account settings page](https://huggingface.co/settings/tokens).

> Add blockquote



In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')
wanbd_token = userdata.get('WANDB')

In [ ]:
from huggingface_hub import login

login(token=hf_token)

In [ ]:
import wandb

wandb.login(wanbd_token)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: leviettin to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import os
os.environ["WANDB_PROJECT"] = "PubMed150"  # name your W&B project

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Dataset

In this step, we load the [**HuggingFaceH4/Multilingual-Thinking**](https://huggingface.co/datasets/HuggingFaceH4/Multilingual-Thinking) dataset from the Hugging Face Hub using the `datasets` library.  
This dataset focuses on **multilingual reasoning**, where the *chain of thought* has been translated into several languages such as French, Spanish, and German.  
By fine-tuning a reasoning-capable model on this dataset, it learns to **generate reasoning steps in multiple languages**, making its thought process more **interpretable and accessible** to non-English speakers.

> 💡 This dataset is best suited for models that already demonstrate reasoning capabilities.  
> If you're using a model without reasoning skills, consider choosing a different dataset. Example: [`trl-lib/llava-instruct-mix`](https://huggingface.co/datasets/trl-lib/llava-instruct-mix).

For efficiency, we'll load only the **training split**:

In [ ]:
import pandas as pd
dataset_name = "JellyFush/PubMedQATropicalDisease"
df = pd.read_csv("/content/drive/MyDrive/LMM/PubMedThinking400.csv")
df

,Index,Source PubMed,Question,Answer,Reasoning,Unnamed: 5
0,1,https://pubmed.ncbi.nlm.nih.gov/26471197/,A 50-year-old male physician is medically evac...,Ebola Virus Disease (complicated by Ebola Inte...,1. Epidemiology: The patient is a healthcare w...,NaN
1,2,https://pmc.ncbi.nlm.nih.gov/articles/PMC7888312/,A 25-year-old male motorcycle taxi driver in t...,Ebola Virus Disease (Relapse),1. Epidemiology/Risk: The patient is a known s...,NaN
2,3,https://pmc.ncbi.nlm.nih.gov/articles/PMC6580501/,"A 22-year-old female, gravida 6 para 4, at 34 ...",Ebola Virus Disease (Maternal Survival with He...,1. Epidemiology: The patient presents in the D...,NaN
3,4,https://pmc.ncbi.nlm.nih.gov/articles/PMC4584529/,A male physician presents to emergency medical...,Ebola Virus Disease,1. Epidemiology/Risk: The patient is a humanit...,NaN
4,5,https://pmc.ncbi.nlm.nih.gov/articles/PMC11464...,A male patient presents to a health facility i...,Ebola Virus Disease (Recrudescent Meningoencep...,1. Epidemiology/Risk: The patient is a survivo...,NaN
...,...,...,...,...,...,...
405,406,https://pubmed.ncbi.nlm.nih.gov/37719554/,"A 23-year-old gravida 5, para 0 Honduran femal...",Strongyloides stercoralis Hyperinfection Syndrome,1. Epidemiology/Risk: The patient is from an e...,NaN
406,407,https://pubmed.ncbi.nlm.nih.gov/36137646/,A 70-year-old male originally from Southeast A...,Strongyloides stercoralis Hyperinfection Syndr...,1. Epidemiology/Risk: The patient is from Sout...,NaN
407,408,https://pubmed.ncbi.nlm.nih.gov/33133837/,A 55-year-old Asian American male presents wit...,Strongyloides stercoralis Hyperinfection Syndrome,1. Epidemiology/Risk: The patient has a critic...,NaN
408,409,https://pubmed.ncbi.nlm.nih.gov/32463606/,A 40-year-old male with a history of homelessn...,Strongyloides stercoralis Hyperinfection Syndrome,1. Epidemiology/Risk: The patient is from an e...,NaN


In [ ]:
# Remove column
df = df.drop(columns=['Unnamed: 5'])
df

,Index,Source PubMed,Question,Answer,Reasoning
0,1,https://pubmed.ncbi.nlm.nih.gov/26471197/,A 50-year-old male physician is medically evac...,Ebola Virus Disease (complicated by Ebola Inte...,1. Epidemiology: The patient is a healthcare w...
1,2,https://pmc.ncbi.nlm.nih.gov/articles/PMC7888312/,A 25-year-old male motorcycle taxi driver in t...,Ebola Virus Disease (Relapse),1. Epidemiology/Risk: The patient is a known s...
2,3,https://pmc.ncbi.nlm.nih.gov/articles/PMC6580501/,"A 22-year-old female, gravida 6 para 4, at 34 ...",Ebola Virus Disease (Maternal Survival with He...,1. Epidemiology: The patient presents in the D...
3,4,https://pmc.ncbi.nlm.nih.gov/articles/PMC4584529/,A male physician presents to emergency medical...,Ebola Virus Disease,1. Epidemiology/Risk: The patient is a humanit...
4,5,https://pmc.ncbi.nlm.nih.gov/articles/PMC11464...,A male patient presents to a health facility i...,Ebola Virus Disease (Recrudescent Meningoencep...,1. Epidemiology/Risk: The patient is a survivo...
...,...,...,...,...,...
405,406,https://pubmed.ncbi.nlm.nih.gov/37719554/,"A 23-year-old gravida 5, para 0 Honduran femal...",Strongyloides stercoralis Hyperinfection Syndrome,1. Epidemiology/Risk: The patient is from an e...
406,407,https://pubmed.ncbi.nlm.nih.gov/36137646/,A 70-year-old male originally from Southeast A...,Strongyloides stercoralis Hyperinfection Syndr...,1. Epidemiology/Risk: The patient is from Sout...
407,408,https://pubmed.ncbi.nlm.nih.gov/33133837/,A 55-year-old Asian American male presents wit...,Strongyloides stercoralis Hyperinfection Syndrome,1. Epidemiology/Risk: The patient has a critic...
408,409,https://pubmed.ncbi.nlm.nih.gov/32463606/,A 40-year-old male with a history of homelessn...,Strongyloides stercoralis Hyperinfection Syndrome,1. Epidemiology/Risk: The patient is from an e...


In [ ]:
json_data = df[:400].to_json(orient='records', lines=True)
json_data
with open('/content/drive/MyDrive/LMM/PubMedThinking400.json', 'w') as f:
    f.write(json_data)

In [ ]:
from datasets import load_dataset

ds = load_dataset("json", data_files="/content/drive/MyDrive/LMM/PubMedThinking400.json", split="train")
ds

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['Index', 'Source PubMed', 'Question', 'Answer', 'Reasoning'],
    num_rows: 400
})

In [ ]:
from datasets import load_dataset

ds_2 = load_dataset("json", data_files="/content/drive/MyDrive/LMM/gemini_clinical_raft_graded.json", split="train")
ds_2

Dataset({
    features: ['id', 'type', 'question', 'answer', 'context', 'oracle_context', 'cot_content', 'cot_answer', 'quality_score'],
    num_rows: 459
})

In [ ]:
ds[0]

{'Index': 1,
 'Source PubMed': 'https://pubmed.ncbi.nlm.nih.gov/26471197/',
 'Question': 'A 50-year-old male physician is medically evacuated to a high-level isolation unit after developing a febrile illness while working in Sierra Leone. His initial clinical course is dominated by severe gastrointestinal symptoms requiring aggressive fluid resuscitation. However, days after admission, his condition deteriorates significantly with the onset of profound hypoxia and respiratory failure, necessitating advanced airway management and mechanical ventilation. Chest imaging reveals diffuse pulmonary infiltrates. Diagnostic bronchoscopy is performed, and molecular testing of the bronchial aspirate reveals an extremely high viral load. Concurrently, laboratory testing identifies a parasitic co-infection. He is treated with experimental antiviral therapies and supportive care, eventually achieving full recovery. What is the most likely diagnosis?',
 'Answer': 'Ebola Virus Disease (complicated by 

In [ ]:
# This is for finetuning with thinking
from datasets import load_dataset
import re

system_prompt = """Role:
You are an expert Medical Diagnostician and Board Examiner.

Task:
Read the provided Clinical Vignette. Determine the most likely diagnosis.
You must first generate a structured clinical reasoning process inside <think> tags, followed by the final diagnosis as plain text.

Output Format:
<think>
[Explanation]
</think>

[Final Diagnosis Name]
"""
def create_messages(row):
    messages = [
        {
            "role": "system",
            "content": system_prompt,
            "thinking": None
        },
        {
            "role": "user",
            "content": row["Question"],
            "thinking": None
        },
        {
            "role": "assistant",
            "content": f"<think>\n{row['Reasoning']}\n</think>\n\n{row['Answer']}",
            "thinking": None
        }
    ]
    row["messages"] = messages
    return row

ds_new = ds.map(create_messages)
ds_new


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Dataset({
    features: ['Index', 'Source PubMed', 'Question', 'Answer', 'Reasoning', 'messages'],
    num_rows: 400
})

In [ ]:
# This is for finetuning with thinking - RAFT
from datasets import load_dataset
import re

system_prompt = """Role:
You are an expert Medical Diagnostician and Board Examiner.

Your task is to answer the user's medical question based STRICTLY on the provided context documents.
1. Analyze the context text provided, check whether the provided document relevant.
2. Provide a Chain of Thought (CoT) reasoning explaining your diagnosis.
3. You must first generate a structured clinical reasoning process inside <think> tags, followed by the final diagnosis as plain text.

Output Format:
<think>
[Explanation]
</think>

[Final Diagnosis Name]
"""
def create_messages(row):
  context_text = ""
  for idx, context in enumerate(row["context"]):
    context_text += f"[Document {idx+1}]:\n{context}\n\n"
  messages = [
      {
          "role": "system",
          "content": system_prompt,
          "thinking": None
      },
      {
          "role": "user",
          "content": row["question"] + "\n\n" + context_text,
          "thinking": None
      },
      {
          "role": "assistant",
          "content": f"<think>\n{row['cot_content']}\n</think>\n\n{row['cot_answer']}",
          "thinking": None
      }
  ]
  row["messages"] = messages
  return row

ds_new_2 = ds_2.map(create_messages)
ds_new_2


Dataset({
    features: ['id', 'type', 'question', 'answer', 'context', 'oracle_context', 'cot_content', 'cot_answer', 'quality_score', 'messages'],
    num_rows: 459
})

In [ ]:
# This is for finetuning with no thinking
from datasets import load_dataset
import re

system_prompt = """Role:
You are an expert Medical Diagnostician and Board Examiner.

Task:
Read the provided Clinical Vignette. Determine the most likely diagnosis.
Output only the disease name.

Output:
[Final Diagnosis Name]
"""
def create_messages(row):
    messages = [
        {
            "role": "system",
            "content": system_prompt,
            "thinking": None
        },
        {
            "role": "user",
            "content": row["Question"],
            "thinking": None
        },
        {
            "role": "assistant",
            "content": f"{row['Answer']}",
            "thinking": None
        }
    ]
    row["messages"] = messages
    return row

ds_new = ds.map(create_messages)
ds_new


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Dataset({
    features: ['Index', 'Source PubMed', 'Question', 'Answer', 'Reasoning', 'messages'],
    num_rows: 400
})

In [ ]:
full_dataset = ds_new.remove_columns(column_names=['Index', 'Source PubMed', 'Question', 'Answer', 'Reasoning'])
full_dataset[0]

{'messages': [{'content': 'Role:\nYou are an expert Medical Diagnostician and Board Examiner.\n\nTask:\nRead the provided Clinical Vignette. Determine the most likely diagnosis.\nYou must first generate a structured clinical reasoning process inside <think> tags, followed by the final diagnosis as plain text.\n\nOutput Format:\n<think>\n[Explanation]\n</think>\n\n[Final Diagnosis Name]\n',
   'role': 'system',
   'thinking': None},
  {'content': 'A 50-year-old male physician is medically evacuated to a high-level isolation unit after developing a febrile illness while working in Sierra Leone. His initial clinical course is dominated by severe gastrointestinal symptoms requiring aggressive fluid resuscitation. However, days after admission, his condition deteriorates significantly with the onset of profound hypoxia and respiratory failure, necessitating advanced airway management and mechanical ventilation. Chest imaging reveals diffuse pulmonary infiltrates. Diagnostic bronchoscopy is 

In [ ]:
full_dataset_raft = ds_new_2.remove_columns(column_names=['id', 'type', 'question', 'answer', 'context', 'oracle_context', 'cot_content', 'cot_answer', 'quality_score'])
full_dataset_raft[0]

{'messages': [{'content': "Role:\nYou are an expert Medical Diagnostician and Board Examiner.\n\nYour task is to answer the user's medical question based STRICTLY on the provided context documents.\n1. Analyze the context text provided, check whether the provided document relevant.\n2. Provide a Chain of Thought (CoT) reasoning explaining your diagnosis.\n3. You must first generate a structured clinical reasoning process inside <think> tags, followed by the final diagnosis as plain text.\n\nOutput Format:\n<think>\n[Explanation]\n</think>\n\n[Final Diagnosis Name]\n",
   'role': 'system',
   'thinking': None},
  {'content': 'A 3-year-old domestic shorthair cat is presented to the emergency clinic with a two-day history of lethargy and decreased appetite. The owner reports the cat was previously active and healthy. Over the past 24 hours, the cat has become increasingly weak and is now exhibiting mild jaundice. On physical examination, the cat is thin and dehydrated. Its mucous membrane

In [ ]:
# This is to split the dataset randomly
# 1. Split the dataset (90% Train, 10% Eval)
# We use a seed to ensure the split is reproducible
# Get first random 100 rows and split
full_dataset_raft = full_dataset_raft.shuffle(seed=42)
full_dataset_raft = full_dataset_raft.select(range(100))
dataset_split_raft = full_dataset_raft.train_test_split(test_size=0.25, seed=42)

train_dataset_raft = dataset_split_raft["train"]
eval_dataset_raft = dataset_split_raft["test"]

# Shuffle the train_dataset
train_dataset_raft = train_dataset_raft.shuffle(seed=42)

print(f"Training samples: {len(train_dataset_raft)}")
print(f"Evaluation samples: {len(eval_dataset_raft)}")

Training samples: 75
Evaluation samples: 25


In [ ]:
# This split the dataset by getting 1 row of each disease and put in the validation set
train_indices = []
val_indices = []

rows_per_disease = 5

for i in range(0, len(full_dataset), rows_per_disease):
    chunk = list(range(i, i + rows_per_disease))

    # 1 sample → validation
    val_indices.append(chunk[0])

    # remaining samples → training
    train_indices.extend(chunk[1:])

train_dataset_sft = full_dataset.select(train_indices)
eval_dataset_sft  = full_dataset.select(val_indices)

train_dataset_sft = train_dataset_sft.shuffle(seed=42)

print(f"Training samples: {len(train_dataset_sft)}")
print(f"Evaluation samples: {len(eval_dataset_sft)}")

Training samples: 320
Evaluation samples: 80


In [ ]:
from datasets import concatenate_datasets

# Check if train_dataset_raft is defined
if 'train_dataset_raft' not in locals():
    train_dataset = train_dataset_sft
else:
    train_dataset = concatenate_datasets([
      train_dataset_raft,
      train_dataset_sft
  ]).shuffle(seed=42)

if 'eval_dataset_raft' not in locals():
    eval_dataset = eval_dataset_sft
else:
    eval_dataset = concatenate_datasets([
      eval_dataset_raft,
      eval_dataset_sft
  ])

print(f"Training samples: {len(train_dataset)}")
print(f"Evaluation samples: {len(eval_dataset)}")

Training samples: 320
Evaluation samples: 80


## Load model and configure LoRA/QLoRA

This notebook can be used with two fine-tuning methods. By default, it is set up for **QLoRA**, which includes quantization using `BitsAndBytesConfig`. If you prefer to use standard **LoRA** without quantization, simply comment out the `BitsAndBytesConfig` configuration.

Below, choose your **preferred model**. All of the options have been tested on **free Colab instances**.

In [ ]:
# Select one model below by uncommenting the line you want to use 👇
## Qwen
# model_id, output_dir = "unsloth/qwen3-14b-unsloth-bnb-4bit", "qwen3-14b-unsloth-bnb-4bit-SFT"     # ⚠️ ~14.1 GB VRAM
# model_id, output_dir = "Qwen/Qwen3-8B", "Qwen3-8B-SFT"                                          # ⚠️ ~12.8 GB VRAM
# model_id, output_dir = "Qwen/Qwen2.5-7B-Instruct", "Qwen2.5-7B-Instruct"                        # ✅ ~10.8 GB VRAM
# model_id, output_dir = "Qwen/Qwen2.5-VL-7B-Instruct", "Qwen2.5-VL-7B-Instruct"
model_id, output_dir = "Qwen/Qwen2.5-VL-3B-Instruct", "Qwen2.5-VL-3B-Instruct"
# model_id, output_dir = "Qwen/Qwen2.5-VL-3B-Instruct", "Qwen2.5-VL-3B-Instruct-No-Reasoning"

## Falcon

## Llama
# model_id, output_dir = "meta-llama/Llama-3.2-3B-Instruct", "Llama-3.2-3B-Instruct"              # ✅ ~4.7 GB VRAM
# model_id, output_dir = "meta-llama/Llama-3.1-8B-Instruct", "Llama-3.1-8B-Instruct"              # ⚠️ ~10.9 GB VRAM

## Gemma
# model_id, output_dir = "google/gemma-3n-E2B-it", "gemma-3n-E2B-it"                              # ❌ Upgrade to a higher tier of colab
# model_id, output_dir = "google/gemma-3-4b-it", "gemma-3-4b-it"                                  # ⚠️ ~6.8 GB VRAM

## Granite
# model_id, output_dir = "ibm-granite/granite-4.0-micro", "granite-4.0-micro"                      # ✅ ~3.3 GB VRAM

## LFM2
#model_id, output_dir = "LiquidAI/LFM2-2.6B", "LFM2-2.6B-SFT"                                     # ✅ ~5.89 GB VRAM

Let's load the selected model using `transformers`, configuring QLoRA via `bitsandbytes` (you can remove it if doing LoRA). We don't need to configure the tokenizer since the trainer takes care of that automatically.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    attn_implementation="sdpa",                   # Change to Flash Attention if GPU has support
    dtype=torch.bfloat16,                          # Change to bfloat16 if GPU has support
    use_cache=True,                               # Whether to cache attention outputs to speed up inference
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,                        # Load the model in 4-bit precision to save memory
        bnb_4bit_compute_dtype=torch.bfloat16,     # Data type used for internal computations in quantization
        bnb_4bit_use_double_quant=True,           # Use double quantization to improve accuracy
        bnb_4bit_quant_type="nf4"                 # Type of quantization. "nf4" is recommended for recent LLMs
    )
)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, BitsAndBytesConfig, AutoTokenizer

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_id,
    attn_implementation="sdpa",                   # Change to Flash Attention if GPU has support
    dtype=torch.bfloat16,                          # Change to bfloat16 if GPU has support
    use_cache=True,                               # Whether to cache attention outputs to speed up inference
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,                        # Load the model in 4-bit precision to save memory
        bnb_4bit_compute_dtype=torch.bfloat16,     # Data type used for internal computations in quantization
        bnb_4bit_use_double_quant=True,           # Use double quantization to improve accuracy
        bnb_4bit_quant_type="nf4"                 # Type of quantization. "nf4" is recommended for recent LLMs
    )
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.53G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

The following cell defines LoRA (or QLoRA if needed). When training with LoRA/QLoRA, we use a **base model** (the one selected above) and, instead of modifying its original weights, we fine-tune a **LoRA adapter** — a lightweight layer that enables efficient and memory-friendly training. The **`target_modules`** specify which parts of the model (e.g., attention or projection layers) will be adapted by LoRA during fine-tuning.

In [ ]:
print(model)

Qwen2_5_VLForConditionalGeneration(
  (model): Qwen2_5_VLModel(
    (visual): Qwen2_5_VisionTransformerPretrainedModel(
      (patch_embed): Qwen2_5_VisionPatchEmbed(
        (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
      )
      (rotary_pos_emb): Qwen2_5_VisionRotaryEmbedding()
      (blocks): ModuleList(
        (0-31): 32 x Qwen2_5_VLVisionBlock(
          (norm1): Qwen2RMSNorm((1280,), eps=1e-06)
          (norm2): Qwen2RMSNorm((1280,), eps=1e-06)
          (attn): Qwen2_5_VLVisionAttention(
            (qkv): Linear4bit(in_features=1280, out_features=3840, bias=True)
            (proj): Linear4bit(in_features=1280, out_features=1280, bias=True)
          )
          (mlp): Qwen2_5_VLMLP(
            (gate_proj): Linear4bit(in_features=1280, out_features=3420, bias=True)
            (up_proj): Linear4bit(in_features=1280, out_features=3420, bias=True)
            (down_proj): Linear4bit(in_features=3420, out_features=1280, bias=True)
        

In [ ]:
from peft import LoraConfig

# You may need to update `target_modules` depending on the architecture of your chosen model.
# For example, different LLMs might have different attention/projection layer names.
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    # lora_dropout=0.05,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    # target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
)

## Train model

We'll configure **SFT** using `SFTConfig`, keeping the parameters minimal so the training fits on a free Colab instance. You can adjust these settings if more resources are available. For full details on all available parameters, check the [TRL SFTConfig documentation](https://huggingface.co/docs/trl/sft_trainer#trl.SFTConfig).

In [ ]:
from trl import SFTConfig

training_args = SFTConfig(
    # Training schedule / optimization
    per_device_train_batch_size = 4,      # Batch size per GPU
    gradient_accumulation_steps = 4,      # Gradients are accumulated over multiple steps → effective batch size = 4 * 8 = 32
    # warmup_steps = 5,
    warmup_ratio=0.1,
    num_train_epochs = 2,               # Number of full dataset passes. For shorter training, use `max_steps` instead (this case)
    max_steps = -1,
    learning_rate = 1e-4,                 # Learning rate for the optimizer
    optim = "paged_adamw_8bit",           # Optimizer
    neftune_noise_alpha = 5,              # Penalize complex weights to prevent hallucination.
    weight_decay = 0.0,                  # (Optional) Adds noise to embeddings. Great for small datasets.

    # Logging / reporting
    logging_steps=1,                      # Log training metrics every N steps
    report_to="wandb",                  # Experiment tracking tool
    # trackio_space_id=output_dir,          # HF Space where the experiment tracking will be saved
    output_dir=output_dir,                # Where to save model checkpoints and logs

    max_length=512,                      # Maximum input sequence length
    use_liger_kernel= False,                # Enable Liger kernel optimizations for faster training
    # activation_offloading=False,           # Offload activations to CPU to reduce GPU memory usage
    gradient_checkpointing=True,          # Save memory by re-computing activations during backpropagation

    # Hub integration
    push_to_hub=True,                     # Automatically push the trained model to the Hugging Face Hub
                                          # The model will be saved under your Hub account in the repository named `output_dir`
    prediction_loss_only=True,
    # Evaluation
    per_device_eval_batch_size=1,         # Batch size for evaluation
    eval_accumulation_steps=1,
    eval_strategy="steps",   # Evaluate every N steps
    eval_steps=5,           # Run evaluation every 5 training steps
    save_strategy="steps",   # Save checkpoints every save_steps
    save_steps=5,          # Save every 5 steps

    gradient_checkpointing_kwargs={"use_reentrant": False}, # To prevent warning message
)

Configure the SFT Trainer. We pass the previously configured `training_args`. We don't use eval dataset to mantain memory usage low but you can configure it.

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config
)

preprocessor_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

chat_template.json: 0.00B [00:00, ?B/s]

Tokenizing train dataset:   0%|          | 0/320 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/320 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/80 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/80 [00:00<?, ? examples/s]

Show memory stats before training

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
3.734 GB of memory reserved.


And train!

In [ ]:
trainer_stats = trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
5,2.608100,2.430007,1.806253,39893.000000,0.529964
10,2.268800,2.124380,1.861272,79555.000000,0.566407
15,2.024800,1.946523,1.942079,119678.000000,0.580376
20,1.835400,1.775922,1.834825,159633.000000,0.598880
25,1.682500,1.651947,1.695191,199578.000000,0.627621
30,1.594900,1.570462,1.617421,239152.000000,0.643289
35,1.543700,1.523850,1.584002,279278.000000,0.648282
40,1.508300,1.509698,1.576692,319266.000000,0.651049


Show memory stats after training

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3870.285 seconds used for training.
64.5 minutes used for training.
Peak reserved memory = 8.391 GB.
Peak reserved memory for training = 4.657 GB.
Peak reserved memory % of max memory = 56.923 %.
Peak reserved memory for training % of max memory = 31.592 %.


The training procedure generates both standard training logs and **trackio** logs, which help us monitor the training progress. Example outputs would look like the following:

![sft-lora-notebook-trackio](https://huggingface.co/datasets/trl-lib/documentation-images/resolve/main/sft-lora-notebook-trackio.png)

## Saving fine tuned model

In this step, we save the fine-tuned model both **locally** and to the **Hugging Face Hub** using the credentials from your account.

In [ ]:
dataset_name = "JellyFush/PubMedTropical"

In [ ]:
from transformers import AutoProcessor

output_dir = "./qwen"
trainer.save_model(output_dir)

if trainer.tokenizer is not None:
    trainer.tokenizer.save_pretrained(output_dir)

processor = None
try:
    processor = AutoProcessor.from_pretrained(model_id, use_fast=True)
except Exception:
    processor = None

if processor is not None and hasattr(processor, "save_pretrained"):
    processor.save_pretrained(output_dir)

trainer.push_to_hub(dataset_name=dataset_name)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...nstruct/training_args.bin: 100%|##########| 6.29kB / 6.29kB            

  ...adapter_model.safetensors:  11%|#1        | 8.39MB / 74.4MB            

  ...B-Instruct/tokenizer.json:  73%|#######2  | 8.30MB / 11.4MB            

...

New Data Upload               : |          |  0.00B /  0.00B            

  ...nstruct/training_args.bin: 100%|##########| 6.29kB / 6.29kB            

  ...B-Instruct/tokenizer.json:  73%|#######2  | 8.30MB / 11.4MB            

  ...adapter_model.safetensors:  11%|#1        | 8.39MB / 74.4MB            

CommitInfo(commit_url='https://huggingface.co/huhyhu/Qwen2.5-VL-3B-Instruct/commit/346a9e1aa954e2c8e9f4ce8f471fac2a18ac7684', commit_message='End of training', commit_description='', oid='346a9e1aa954e2c8e9f4ce8f471fac2a18ac7684', pr_url=None, repo_url=RepoUrl('https://huggingface.co/huhyhu/Qwen2.5-VL-3B-Instruct', endpoint='https://huggingface.co', repo_type='model', repo_id='huhyhu/Qwen2.5-VL-3B-Instruct'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoProcessor

trainer.save_model(output_dir)
trainer.push_to_hub(dataset_name=dataset_name)
processor = AutoProcessor.from_pretrained(model_id)
processor.save_pretrained(output_dir)
processor.push_to_hub(output_dir)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...nstruct/training_args.bin: 100%|##########| 6.29kB / 6.29kB            

  ...adapter_model.safetensors:  23%|##2       | 16.7MB / 74.4MB            

  ...B-Instruct/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            

...

  ...adapter_model.safetensors:  23%|##2       | 16.7MB / 74.4MB            

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpzdsr5djl/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            

CommitInfo(commit_url='https://huggingface.co/JellyFush/PubMedTropical/commit/6a8d1a289b8233d5bf6faa8df4c629d82c2ed0bc', commit_message='Upload processor', commit_description='', oid='6a8d1a289b8233d5bf6faa8df4c629d82c2ed0bc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/JellyFush/PubMedTropical', endpoint='https://huggingface.co', repo_type='model', repo_id='JellyFush/PubMedTropical'), pr_revision=None, pr_num=None)

## Load the fine-tuned model and run inference

Now, let's test our fine-tuned model by loading the **LoRA/QLoRA adapter** and performing **inference**. We'll start by loading the **base model**, then attach the adapter to it, creating the final fine-tuned model ready for evaluation.

In [ ]:
model_id, output_dir = "Qwen/Qwen2.5-VL-3B-Instruct", "Qwen2.5-VL-3B-Instruct"
# model_id, output_dir = "Qwen/Qwen2.5-VL-3B-Instruct", "Qwen2.5-VL-3B-Instruct-No-Reasoning"
# model_id, output_dir = "meta-llama/Llama-3.2-3B-Instruct", "Llama-3.2-3B-Instruct"
# model_id, output_dir = "meta-llama/Llama-3.2-3B-Instruct", "Llama-3.2-3B-Instruct-MedCaseReason"

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from peft import PeftModel


base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    attn_implementation="sdpa",                   # Change to Flash Attention if GPU has support
    dtype=torch.float16,                          # Change to bfloat16 if GPU has support
    device_map="auto",
    # use_cache=True,                               # Whether to cache attention outputs to speed up inference
    # quantization_config=BitsAndBytesConfig(
    #     load_in_4bit=True,                        # Load the model in 4-bit precision to save memory
    #     bnb_4bit_compute_dtype=torch.float16,     # Data type used for internal computations in quantization
    #     bnb_4bit_use_double_quant=True,           # Use double quantization to improve accuracy
    #     bnb_4bit_quant_type="nf4"                 # Type of quantization. "nf4" is recommended for recent LLMs
    # )
)

adapter_model = f"JellyFush/{output_dir}" # Replace with your HF username or organization

tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, BitsAndBytesConfig, AutoTokenizer
from peft import PeftModel


base_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_id,
    attn_implementation="sdpa",                   # Change to Flash Attention if GPU has support
    dtype=torch.float16,                          # Change to bfloat16 if GPU has support
    device_map="auto",
    # use_cache=True,                               # Whether to cache attention outputs to speed up inference
    # quantization_config=BitsAndBytesConfig(
    #     load_in_4bit=True,                        # Load the model in 4-bit precision to save memory
    #     bnb_4bit_compute_dtype=torch.float16,     # Data type used for internal computations in quantization
    #     bnb_4bit_use_double_quant=True,           # Use double quantization to improve accuracy
    #     bnb_4bit_quant_type="nf4"                 # Type of quantization. "nf4" is recommended for recent LLMs
    # )
)

adapter_model = f"JellyFush/{output_dir}" # Replace with your HF username or organization

tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.53G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Let's create a sample message using the dataset's structure. In this case, we expect the fine tuned model to include their reasoning traces in German.

In [ ]:
messages = [
  {
      'content': """Role:
You are an expert Medical Diagnostician and Board Examiner.

Task:
Read the provided Clinical Vignette. Determine the most likely diagnosis.
You must first generate a structured clinical reasoning process inside <think> tags, followed by the final diagnosis as plain text.

Output Format:
<think>
[Explanation]
</think>

[Final Diagnosis Name]
""",
      'role': 'system',
  },
  {
      'content': """A 6-year-old boy, known to be HIV-positive but not yet on antiretroviral therapy (ART), presents with acute onset of high fever, dyspnoea, dry cough, and impaired consciousness for 3 days. He has coryza, bilateral conjunctivitis, and a fine maculopapular rash behind both ears and on the forehead that later covers the trunk with skin scaling over the chest. He also has bilateral axillary lymphadenopathy and laboured breathing. Malaria smears were negative. He is from Malawi.
      """,
      'role': 'user',
  }
]

In [ ]:
messages = [
  {
      'content': """Role:
You are an expert Medical Diagnostician and Board Examiner.

Task:
Read the provided Clinical Vignette. Determine the most likely diagnosis.
Output only the disease name.

Output:
[Final Diagnosis Name]
""",
      'role': 'system',
  },
  {
      'content': """A 6-year-old boy, known to be HIV-positive but not yet on antiretroviral therapy (ART), presents with acute onset of high fever, dyspnoea, dry cough, and impaired consciousness for 3 days. He has coryza, bilateral conjunctivitis, and a fine maculopapular rash behind both ears and on the forehead that later covers the trunk with skin scaling over the chest. He also has bilateral axillary lymphadenopathy and laboured breathing. Malaria smears were negative. He is from Malawi.
      """,
      'role': 'user',
  }
]

Let's first check what's the output for the base model, without the adapter.

In [ ]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(base_model.device)

generated_ids = base_model.generate(
    **model_inputs,
    max_new_tokens=512
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):]

# Decode and extract model response
generated_text = tokenizer.decode(output_ids, skip_special_tokens=True)
print(generated_text)

<think>
The patient's symptoms of high fever, dyspnoea, dry cough, impaired consciousness, coryza, bilateral conjunctivitis, a fine maculopapular rash, bilateral axillary lymphadenopathy, and labored breathing are highly suggestive of a viral infection. The presence of bilateral axillary lymphadenopathy and the rash suggests a viral exanthem. Given the patient's history of HIV infection, it is important to consider opportunistic infections, particularly those caused by viruses. The combination of these symptoms, especially the rash and lymphadenopathy, is characteristic of Kaposi's sarcoma-associated herpesvirus (KSHV) infection, which can present in HIV-positive individuals as Kaposi's sarcoma or other viral infections such as Epstein-Barr virus (EBV) infection.

Malaria smears being negative rules out malaria as the cause of the symptoms. The rash and lymphadenopathy are more consistent with EBV infection, which can occur in HIV-positive individuals due to immunosuppression. However,

We can see that the reasoning traces are in English, which is expected. Let's now load the fine-tuned model and check its answer.

In [ ]:
fine_tuned_model = PeftModel.from_pretrained(base_model, adapter_model)

adapter_config.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/74.4M [00:00<?, ?B/s]

In [ ]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(fine_tuned_model.device)

generated_ids = fine_tuned_model.generate(
    **model_inputs,
    max_new_tokens=2048
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):]

# Decode and extract model response
generated_text = tokenizer.decode(output_ids, skip_special_tokens=True)
print(generated_text)

<think>
1. **Clinical Presentation**: The patient presents with a high fever, respiratory distress, cough, and a distinctive rash (maculopapular rash with scaling) in a setting of HIV infection. The rash is a key diagnostic feature.
2. **Differential Diagnosis**:
   - **Malaria**: While malaria was ruled out by negative smears, the clinical presentation is atypical for malaria in this setting.
   - **Acute Exanthematous Pustulosis (AEP)**: This is a drug eruption caused by certain antibiotics, often associated with fever, rash, and lymphadenopathy. Given the history of recent antibiotic use (likely for a respiratory infection), this could be a plausible diagnosis.
   - **Erythema Multiforme (EM)**: A severe cutaneous reaction to various triggers, including infections, can present with a maculopapular rash, fever, and lymphadenopathy. However, EM typically does not cause respiratory distress or have a specific rash pattern.
   - **Drug Reaction**: Given the recent antibiotic use, a drug

The model now generates its reasoning trace in German!

## Inference and Serving with vLLM

You can use Transformer models with **vLLM** to serve them in real-world applications. Learn more [here](https://blog.vllm.ai/2025/04/11/transformers-backend.html).

In [ ]:
!pip install -qU vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.9/474.9 MB 796.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.0/355.0 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 128.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 122.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 137.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.

### Push Merged Model (for LoRA or QLoRA Training)

To serve the model via **vLLM**, the repository must contain the merged model (base model + LoRA adapter). Therefore, you need to upload it first.

In [ ]:
model_merged = fine_tuned_model.merge_and_unload()

save_dir = f"{output_dir}-merged"

model_merged.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('Llama-3.2-3B-Instruct-merged/tokenizer_config.json',
 'Llama-3.2-3B-Instruct-merged/special_tokens_map.json',
 'Llama-3.2-3B-Instruct-merged/chat_template.jinja',
 'Llama-3.2-3B-Instruct-merged/tokenizer.json')

In [ ]:
model_merged.push_to_hub(f"JellyFush/{output_dir}-merged") # Replace with your HF username or organization
tokenizer.push_to_hub(f"JellyFush/{output_dir}-merged") # Replace with your HF username or organization

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0001-of-00002.safetensors:   0%|          | 8.28MB / 4.97GB            

  ...0002-of-00002.safetensors:   2%|1         | 25.2MB / 1.46GB            

...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...uct-merged/tokenizer.json:   2%|1         |  295kB / 17.2MB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/JellyFush/Llama-3.2-3B-Instruct-merged/commit/584fbeda1e4515a7201b6da1cdeb223cc62363a1', commit_message='Upload tokenizer', commit_description='', oid='584fbeda1e4515a7201b6da1cdeb223cc62363a1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/JellyFush/Llama-3.2-3B-Instruct-merged', endpoint='https://huggingface.co', repo_type='model', repo_id='JellyFush/Llama-3.2-3B-Instruct-merged'), pr_revision=None, pr_num=None)

### Performing Inference with vLLM

Use **vLLM** to run your model and generate text efficiently in real-time. This allows you to test and deploy your fine-tuned models with low latency and high throughput.

In [ ]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import torch

llm = LLM(
    model=f"sergiopaniego/{output_dir}-merged", # Replace with your HF username or organization
    model_impl="transformers",                  # Select the transformers model implementation
    max_model_len=512,                         # Reduced for efficiency
    dtype=torch.float16
)
hf_tokenizer = AutoTokenizer.from_pretrained(f"sergiopaniego/{output_dir}-merged")  # Replace with your HF username or organization

In [ ]:
# Alternatively, use llm.chat()
prompt = hf_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

outputs = llm.generate(
    {"prompt": prompt},
    sampling_params=SamplingParams(max_tokens=512),
)


for o in outputs:
    generated_text = o.outputs[0].text
    print(generated_text)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

<think>
Mag nachdenken...igkeit. Ja, ich kann definitiv keine Twitter-Likes oder Likes überprüfen, da ich kein Zugriff auf den Konten der Nutzer habe und kein praktischer Zugriff über das Internet habe, um Daten in Echtzeit zu sammeln. Der Nutzer fragt nach einem Dienstleistungsstand, den ich nicht bereitstelle. Ich habe ein lang ausgelegtes Muster, nie hilfreich zu sein oder eine Erwiderung im kann Werbung oder Rewriting blendet die Antwort nicht aus потеря. Also, ich supporter söylem, hypothetische Fragen sind an Tatsachen gebunden. Ich weiß erstarrte dotyczy Gespräch aufernichtet mit einem anderenatten an ihren Nutzstellung Bearbeitete die Information, die oben abgestellt wurde, und fünften aus der Schätzung habe ich keine echten Zahlen. Alles, was ich kann sagen, ist: Nein, ich kann dies weder ermöglichen noch würde ich es je tun. In dem Sinne, 然后 ich wähle vor der Available antwortem, remains in das 'No' Verkleidung an,optiґxt; Alles, was ich zum Eintritt in den Band Emblem curve,